Task:

PFA

A person’s signature is representative of his identity. For us at the Bank, a signed document by a customer is an instruction from him for carrying out an approved transaction for him.

On on-boarding a customer we capture an image of his signature in our systems, and on receiving a signed document (Cheques, DDs, and others) from him we match the signature on the document with the one recorded in the database before proceeding with the instruction.

In the case of skilled forgeries, it becomes very difficult to verify the identity of the customer.

We want you to build a system that can help us distinguish forgeries from actual signatures. This system should be able to study signature parameters as strokes, curves, dots, dashes, writing fluidity & style, in a Writer-Independent manner and create features for identification of the signature.

The system should not use any existing APIs and should be completely self-developed.

How should it work?

The system shall work in 2 steps:

Step 1: Accept & Store Genuine Signature Image: Take actual signature scanned image of the onboarding customer and store it in a database against a unique Customer ID

Step 2: Accept & Compare Signature Images: Accept inputs of Customer ID and corresponding signature image. Compare with the signature stored in DB against the given Customer ID, and return a Confidence Match Score between the two signature images.

In [ ]:
# Import all the necessary Library 
import torchvision
import torch.utils.data as utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import time
import copy
from torch.optim import lr_scheduler
import os
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd 
from pathlib import Path

use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not use_cuda:
    print('No GPU found. Please use a GPU to train your neural network.')

In [ ]:
class SignaturesNetwork(nn.Module):
    def __init__(self):
        super(SignaturesNetwork, self).__init__()
        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
        nn.Conv2d(1, 96, kernel_size=11,stride=1),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(3, stride=2),

        nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(3, stride=2),
        nn.Dropout2d(p=0.3),

        nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(3, stride=2),
        nn.Dropout2d(p=0.3),
        )

        # Defining the fully connected layers
        self.fc1 = nn.Sequential(
        # First Dense Layer
        nn.Linear(30976, 1024),
        nn.ReLU(inplace=True),
        nn.Dropout2d(p=0.5),
        # Second Dense Layer
        nn.Linear(1024, 128),
        nn.ReLU(inplace=True),
        # Final Dense Layer
        nn.Linear(128,2))

    def forward_once(self, x):
        # Forward pass 
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.forward_once(input1)
        # forward pass of input 2
        output2 = self.forward_once(input2)
        # returning the feature vectors of two inputs
        return output1, output2


In [ ]:
"""
https://hackernoon.com/facial-similarity-with-siamese-networks-in-pytorch-9642aa9db2f7
"""
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, output1, output2, label):
        # Find the pairwise distance or eucledian distance of two output feature vectors
        euclidean_distance = F.pairwise_distance(output1, output2)
        # perform contrastive loss calculation with the distance
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) + 
        (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        
        return loss_contrastive

In [ ]:
from PIL import ImageEnhance

class SignaturesDataset():
    
    def __init__(self, train, genuine_path, training_dir=None, transform=None, enhance_factor=4.5):
        # used to prepare the labels and images path
        self.training_df = train
        self.columns = ["file","signed_file","status"]
        self.training_dir = training_dir    
        self.transform = transform
        self.genuine_path = genuine_path
        self.enhance_factor = enhance_factor

    def _enhance(self, image):
        enhancer = ImageEnhance.Sharpness(image)
        return enhancer.enhance(self.enhance_factor)

    def __getitem__(self,index):
        
        # getting the image path
        original_path=os.path.join(
            self.genuine_path,
            self.training_df.iat[index, 0])

        compare_path=os.path.join(
            self.training_dir,
            self.training_df.iat[index, 1])
        
        target = torch.from_numpy(
            np.array([int(self.training_df.iat[index, 2])],dtype=np.float32))

        # Loading the image
        original_img = self._enhance(Image.open(original_path))
        compare_img = self._enhance(Image.open(compare_path))

        original_img = original_img.convert("L")
        compare_img = compare_img.convert("L")

        #original_img = np.array(original_img)
        #compare_img = np.array(compare_img)

        data_transform = transforms.Compose([#transforms.ToPILImage(),
                                            transforms.Resize((105,105), Image.ANTIALIAS),
                                            #transforms.Grayscale(1),
                                            #transforms.RandomResizedCrop(224),
                                            #transforms.RandomHorizontalFlip(),
                                            #transforms.RandomVerticalFlip(),
                                            #transforms.RandomRotation(degrees=40), 
                                            transforms.ToTensor(),
                                            #transforms.Normalize(mean=[0.456], std=[0.224])
                                            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                            ])
        
        test_transform = transforms.Compose([#transforms.ToPILImage(),
                                            transforms.Resize((105,105), Image.ANTIALIAS),
                                            #transforms.Grayscale(1),
                                            #transforms.Resize(256),
                                            #transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            #transforms.Normalize(mean=[0.456], std=[0.224])
                                            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                            ])
        
        # Apply image transformations
        if self.transform == "train":
            original_img = data_transform(original_img)
            compare_img = data_transform(compare_img)
        else:
            original_img = test_transform(original_img)
            compare_img = test_transform(compare_img)
        
        return original_img, compare_img , target
    
    def __len__(self):
        return len(self.training_df)

In [ ]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
genuine_path = Path('/kaggle/input/signature-verification-dataset/sign_data/sign_data/train')
forged_path = Path('/kaggle/input/signature-verification-dataset/sign_data/sign_data/test')
path = Path('/kaggle/input/signature-verification-dataset/sign_data/sign_data/')

train = pd.read_csv(path/"train_data.csv", low_memory=False)
test = pd.read_csv(path/"test_data.csv", low_memory=False)
display(train.head())

In [ ]:
# Load the the dataset from raw image folders

#an = SignaturesDataset(train, genuine_path)
#attrs = vars(an)
#print(attrs)

signatures_dataset = SignaturesDataset(train, genuine_path, training_dir=genuine_path, transform="train")
signatures_dataset_test = SignaturesDataset(test, forged_path, training_dir=forged_path, transform="test")

In [ ]:
# Viewing the sample of images and to check whether its loading properly
vis_dataloader = DataLoader(signatures_dataset,
                        shuffle=True,
                        batch_size=8)

dataiter = iter(vis_dataloader)


example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0], example_batch[1]), 0)
imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy())

In [ ]:
batch_size = 32
epochs = 40
learning_rate = 1e-4
alpha = 0.99
show_every_n_batches = 5

In [ ]:
# Load the dataset as pytorch tensors using dataloader
train_dataloader = DataLoader(signatures_dataset,
                        shuffle=True,
                        num_workers=8,
                        batch_size=batch_size)

test_dataloader = DataLoader(signatures_dataset_test, num_workers=6, batch_size=1, shuffle=True)

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    if type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
# Declare Siamese Network
net = SignaturesNetwork()
#net = SiameseNetwork()

net.apply(init_weights)

if use_cuda:
    net = net.to('cuda')
    net = torch.nn.DataParallel(net)

# Decalre Loss Function
margin = 2
criterion = ContrastiveLoss(margin)
# Declare Optimizer
optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, alpha=alpha, eps=1e-8, weight_decay=0.0005, momentum=0.9)
#optimizer = optim.Adam(net.parameters(),lr = learning_rate )

In [ ]:
def train():
    counter = []
    loss_history = []
    iteration_number= 0
    original_loss = np.inf
    valid_loss = 0
    total_loss = 0
    
    for epoch in range(epochs):
        
        for i, (img0, img1 , label) in enumerate(train_dataloader):
            #img0, img1 , label = data
            img0, img1 , label = img0.to('cuda', non_blocking=True), img1.to('cuda', non_blocking=True) , label.to('cuda', non_blocking=True)
            
            optimizer.zero_grad()
            output1, output2 = net(img0, img1)
            loss_contrastive = criterion(output1, output2, label)
            loss_contrastive.backward()
            optimizer.step()
            
            total_loss += loss_contrastive.item() * img0.size(0)
            #valid_corrects += torch.sum(preds == target.data)
            iteration_number += 1
            
        total_loss = total_loss/len(train_dataloader.dataset)
        
        if epoch%show_every_n_batches == 0:
            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, total_loss, valid_loss))
            counter.append(iteration_number)
            loss_history.append(total_loss)
            if total_loss < original_loss:
                print('Training loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(original_loss,
                total_loss))
                with open('model.pt', 'wb') as pickle_file:
                    torch.save(net.state_dict(), pickle_file)
                original_loss = total_loss
            total_loss = 0
    show_plot(counter, loss_history)
    return net

In [ ]:
model = train()
torch.save(model.state_dict(), "model.pt")
print("Model Saved Successfully")

[my Model](model.pt)

In [ ]:
model = net
model.load_state_dict(torch.load("model.pt"))

In [ ]:
accuracy=0
counter=0
correct=0

for i, data in enumerate(test_dataloader,0):
    
    x0, x1, label = data    
    # onehsot applies in the output of 128 dense vectors which is then converted to 2 dense vectors
    
    output1, output2 = model(x0.to(device),x1.to(device))
    
    res = nn.Softmax(dim=1)(output1.cuda() - output2.cuda())
    label = label[0].tolist()
    label = int(label[0])
    result=torch.max(res,1)[1].data[0].tolist()
    if label == result:
        correct = correct+1
    counter=counter+1
    accuracy=(correct/len(test_dataloader))*100

print("Accuracy:{}%".format(accuracy))

In [ ]:
# Print the sample outputs to view its dissimilarity
counter=0
list_1 = torch.FloatTensor([[1]])
list_0 = torch.FloatTensor([[0]])

for i, data in enumerate(test_dataloader,0):
    x0, x1, label = data
    concatenated = torch.cat((x0,x1),0)
    output1, output2 = model(Variable(x0).to(device),Variable(x1).to(device))
    
    res = nn.Softmax(dim=1)(output1.cuda() - output2.cuda())
    eucledian_distance_1 = F.pairwise_distance(output2, output1)
    #eucledian_distance_2 = (output2 - output1).pow(2).sum(1)
    result=torch.max(res,1)[1].data[0].tolist()
    print(res)
    #print(eucledian_distance_1.item())
    #print(eucledian_distance_2)
    
    if result==1:
        pred_label="Orginial"
    else:
        pred_label="Forged"
        
    if label==list_1:
        label="Orginial"
    else:
        label="Forged"
    
    imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f} Label: {} predicted: {}'.format(eucledian_distance_1.item(),label, pred_label))
    counter = counter + 1
    if counter ==20:
        break